In [2]:
import numpy as np
import json
import os

In [5]:
def load_divisions_from_files():
    # Define the directory and filename
    directory = '../data/parliament'
    filename = f"{directory}/divisions.json"
    
    # Check if the file with all divisions exists
    if os.path.exists(filename):
        # Load data from the file if it exists
        with open(filename, 'r', encoding='utf-8') as f:
            return json.load(f)
    else:
        #st.write("No local division data found.")
        return None

In [8]:
divisions = load_divisions_from_files()
senate_divisions = {}

In [10]:
for division_id, division_data in divisions.items():
    if division_data['house'] == 'senate':
        senate_divisions[division_id] = division_data

In [26]:
# Initialize a set to hold unique senator IDs
unique_senator_ids = set()

# Iterate through senate_divisions to collect unique senator IDs
for division_id, division_data in senate_divisions.items():
    for vote_data in division_data['votes']:
        senator_id = vote_data['member']['id']
        unique_senator_ids.add(senator_id)

# Determine the number of unique senators
num_of_senators = len(unique_senator_ids)

# Initialize the interaction matrix
interaction_matrix = np.zeros((num_of_senators, num_of_senators))

# Create a mapping from senator IDs to matrix indices
senator_id_to_index = {senator_id: index for index, senator_id in enumerate(unique_senator_ids)}


In [31]:
# Iterate through each division
for division_id, division_data in senate_divisions.items():  
    votes = {vote_data['member']['id']: vote_data['vote'] for vote_data in division_data['votes']}
    
    # Update the matrix
    for senator_id1 in unique_senator_ids:
        for senator_id2 in unique_senator_ids:
            if senator_id1 == senator_id2:
                continue  # Skip self-comparison
            
            # Map senator IDs to matrix indices
            i = senator_id_to_index[senator_id1]
            j = senator_id_to_index[senator_id2]
            
            # Check if both senators are in the current division
            if senator_id1 in votes and senator_id2 in votes:
                if votes[senator_id1] == votes[senator_id2]:
                    if votes[senator_id1] != 'Absent':  # Skip if both senators are Absent
                        interaction_matrix[i][j] += 1
                        interaction_matrix[j][i] += 1  # The matrix is symmetric

# Normalize the matrix (optional)
total_divisions = len(senate_divisions)  # Replace with the actual number of divisions if different
interaction_matrix = interaction_matrix / total_divisions


In [28]:
len(interaction_matrix)

78

In [32]:
interaction_matrix

array([[0.        , 0.61096606, 0.36553525, ..., 0.11488251, 1.0078329 ,
        1.32114883],
       [0.61096606, 0.        , 0.38642298, ..., 0.2924282 , 0.67885117,
        0.81462141],
       [0.36553525, 0.38642298, 0.        , ..., 0.845953  , 0.32375979,
        0.46997389],
       ...,
       [0.11488251, 0.2924282 , 0.845953  , ..., 0.        , 0.10966057,
        0.15665796],
       [1.0078329 , 0.67885117, 0.32375979, ..., 0.10966057, 0.        ,
        1.32114883],
       [1.32114883, 0.81462141, 0.46997389, ..., 0.15665796, 1.32114883,
        0.        ]])

In [33]:
# Convert the NumPy array to a list of lists
interaction_matrix_list = interaction_matrix.tolist()

# Define the path to save the JSON file
save_path = "../data/parliament/senate_interaction_matrix.json"

# Save to JSON
with open(save_path, 'w') as f:
    json.dump(interaction_matrix_list, f)

print(f"Interaction matrix saved to {save_path}")

Interaction matrix saved to ../data/parliament/senate_interaction_matrix.json


In [34]:
# Save unique_senator_ids to JSON
with open("../data/parliament/unique_senator_ids.json", 'w') as f:
    json.dump(list(unique_senator_ids), f)

# Save senator_id_to_index to JSON
with open("../data/parliament/senator_id_to_index.json", 'w') as f:
    json.dump(senator_id_to_index, f)
